## Probabilistic parsing

In [1]:
import nltk

In [2]:
grammar = nltk.CFG.fromstring("""
    S -> NP V NP
    NP -> NP Sbar
    Sbar -> NP V
    NP -> 'fish'
    V -> 'fish'
""")

In [3]:
tokens = ["fish"] * 3
cp = nltk.ChartParser(grammar)
for tree in cp.parse(tokens):
    print(tree)

(S (NP fish) (V fish) (NP fish))


### A probabilistic context-free grammar

In [4]:
grammar = nltk.PCFG.fromstring("""
    S    -> NP VP              [1.0]
    VP   -> TV NP              [0.4]
    VP   -> IV                 [0.3]
    VP   -> DatV NP NP         [0.3]
    TV   -> 'saw'              [1.0]
    IV   -> 'ate'              [1.0]
    DatV -> 'gave'             [1.0]
    NP   -> 'telescopes'       [0.8]
    NP   -> 'Jack'             [0.2]
    """)

In [5]:
print(grammar)

Grammar with 9 productions (start state = S)
    S -> NP VP [1.0]
    VP -> TV NP [0.4]
    VP -> IV [0.3]
    VP -> DatV NP NP [0.3]
    TV -> 'saw' [1.0]
    IV -> 'ate' [1.0]
    DatV -> 'gave' [1.0]
    NP -> 'telescopes' [0.8]
    NP -> 'Jack' [0.2]


In [6]:
viterbi_parser = nltk.ViterbiParser(grammar)

In [7]:
for tree in viterbi_parser.parse(['Jack', 'saw', 'telescopes']):
    print(tree)

(S (NP Jack) (VP (TV saw) (NP telescopes))) (p=0.064)


In [8]:
0.2*0.4*0.8

0.06400000000000002

In [9]:
for tree in viterbi_parser.parse(['Jack', 'ate']):
    print(tree)

(S (NP Jack) (VP (IV ate))) (p=0.06)


## Solution to exercise

In [10]:
#extend to accommodate dative verbs, e.g. Jack gave Jill telescopes
pp_grammar = nltk.PCFG.fromstring("""
    S    -> NP VP              [1.0]
    VP   -> TV NP              [0.4]
    VP   -> IV                 [0.3]
    VP   -> DatV NP NP         [0.3]
    TV   -> 'saw'              [1.0]
    IV   -> 'ate'              [1.0]
    DatV -> 'gave'             [1.0]
    NP   -> 'telescopes'       [0.8]
    NP   -> 'Jack'             [0.1]
    NP   -> 'Jill'             [0.1]  
    """)

In [11]:
viterbi_parser = nltk.ViterbiParser(pp_grammar)
for tree in viterbi_parser.parse(['Jack', 'gave', 'Jill', 'telescopes']):
     print(tree)

(S (NP Jack) (VP (DatV gave) (NP Jill) (NP telescopes))) (p=0.0024)


In [12]:
#extend to accommodate NP->N
pp_grammar = nltk.PCFG.fromstring("""
    S    -> NP VP              [1.0]
    VP   -> TV NP              [0.4]
    VP   -> IV                 [0.3]
    VP   -> DatV NP NP         [0.3]
    TV   -> 'saw'              [1.0]
    IV   -> 'ate'              [1.0]
    DatV -> 'gave'             [1.0]
    NP   -> N                  [1.0]
    N   -> 'telescopes'        [0.8]
    N   -> 'Jack'              [0.1]
    N   -> 'Jill'              [0.1]  
    """)

In [13]:
#check it still works
viterbi_parser = nltk.ViterbiParser(pp_grammar)
for tree in viterbi_parser.parse(['Jack', 'gave', 'Jill', 'telescopes']):
     print(tree)

(S
  (NP (N Jack))
  (VP (DatV gave) (NP (N Jill)) (NP (N telescopes)))) (p=0.0024)


In [14]:
#extend to accommodate PPs, e.g. Jack gave telescopes to Jill
pp_grammar = nltk.PCFG.fromstring("""
    S    -> NP VP              [1.0]
    VP   -> TV NP              [0.4]
    VP   -> IV                 [0.3]
    VP   -> DatV NP NP         [0.2]
    VP   -> DatV NP PP         [0.1]
    TV   -> 'saw'              [1.0]
    IV   -> 'ate'              [1.0]
    DatV -> 'gave'             [1.0]
    NP   -> PP N               [0.5]
    NP   -> N                  [0.5]
    PP   -> P N                [1.0]
    P    -> 'to'               [1.0]
    N    -> 'telescopes'       [0.8]
    N    -> 'Jack'             [0.1]
    N    -> 'Jill'             [0.1]  
    """)

In [15]:
#check it still works
viterbi_parser = nltk.ViterbiParser(pp_grammar)
for tree in viterbi_parser.parse(['Jack', 'gave', 'Jill', 'telescopes']):
     print(tree)

(S
  (NP (N Jack))
  (VP (DatV gave) (NP (N Jill)) (NP (N telescopes)))) (p=0.0002)


In [16]:
#test
viterbi_parser = nltk.ViterbiParser(pp_grammar)
for tree in viterbi_parser.parse(['Jack', 'gave', 'telescopes', 'to', 'Jill']):
     print(tree)

(S
  (NP (N Jack))
  (VP (DatV gave) (NP (N telescopes)) (PP (P to) (N Jill)))) (p=0.0002)


In [17]:
#final exercise: 
#adjust probabilities to suit your intuitions
#discussion points:
#what kinds of syntactic construction did you add to accommodate these extensions?
#how did you estimate their respective probabilities?
#how do you think such probabilities would be estimated in practice?